# **EDA (Exploratory Data Analysis)** de `The Movies Dataset`

## Data Collection

### **Cargar los datos ([`The Movies Dataset`](https://grouplens.org/datasets/movielens/))**


In [ ]:
import zipfile
import os
# descarmaos usando el CLI de kaggle
!kaggle datasets download -d rounakbanik/the-movies-dataset -p data/
# descomprimimos el archivo
with zipfile.ZipFile("./raw/the-movies-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("./raw")
# removemos el archivo zip
os.remove("./raw/the-movies-dataset.zip")
# removemos los datasets miniatura<
os.remove("./raw/links_small.csv")
os.remove("./raw/ratings_small.csv")

No se usaran los dataset `ratings.csv` y `links.csv` para nuestro caso.

In [11]:
import pandas as pd
# off warnings
import warnings
warnings.filterwarnings("ignore")

df_credits = pd.read_csv("./raw/credits.csv")
df_keywords = pd.read_csv("./raw/keywords.csv")
df_movies_metada = pd.read_csv("./raw/movies_metadata.csv")
df_ratings = pd.read_csv("./raw/ratings.csv")

In [2]:
# cambiar ancho de output del notebook
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", 80)

`ratings.csv`

In [4]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


#### `credits.csv`


In [3]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


#### `keywords.csv`


In [4]:
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


#### `movies_metadata.csv`


In [5]:
df_movies_metada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

## Data Exploration

- Revisar los tipos de datos
- Revisar los valores nulos
- Revisar los valores duplicados
- Revisar las estadísticas básicas


`ratings.csv`

In [5]:
df_ratings.shape

(26024289, 4)

In [6]:
df_ratings.duplicated().sum()

np.int64(0)

#### `credits.csv`


In [6]:
# Revisamos la cantidad de columnas y filas
df_credits.shape

(45476, 3)

In [7]:
# Revisamos las datos duplicados
df_credits.duplicated().sum()

np.int64(37)

#### `keywords.csv`


In [8]:
# Revisamos la cantidad de columnas y filas
df_keywords.shape

(46419, 2)

In [9]:
# Revisamos los valores nulos
df_keywords.isnull().sum()

id          0
keywords    0
dtype: int64

In [10]:
# Revisamos las datos duplicados
df_keywords.duplicated().sum()

np.int64(987)

#### `movies_metadata.csv`


In [11]:
# Revisamos la cantidad de columnas y filas
df_movies_metada.shape

(45466, 24)

In [12]:
# Revisamos los valores nulos
df_movies_metada.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [13]:
# Revisamos las datos duplicados
df_movies_metada.duplicated().sum()

np.int64(13)

## Data Preprocessing
- Eliminar columnas innecesarias
- Eliminar registros con valores nulos
- Eliminar registros duplicados
- Convertir los tipos de datos

Creamos una funcion para limpiar los datos


In [14]:
from ast import literal_eval
from typing import Callable

# funcion para obtener los valores de un string
def extract_info(text, obj: str = "name", func: Callable = lambda _: True):
    # Si el texto es un string sin nada que evaluar
    try:
        text = literal_eval(text)
    except:
        return text
    # Si el texto es un diccionario
    if type(text) == dict:
        return text[obj]
    # Si el texto es una lista
    if type(text) == list:
        if len(text) == 0: # Si la lista esta vacia
            return "Others"
        if len(text) == 1: # Si la lista tiene un solo elemento
            for i in text:
                return i[obj]
        else: # Si la lista tiene mas de un elemento
            s = [] # lista para almacenar los valores
            for i in text:
                if func(i):  # si la funcion es verdadera
                    s.append(i[obj]) # agregamos el valor a la lista
            return ", ".join(s) # retornamos la lista como un string

#### `credits.csv`


In [15]:
df_credits.shape

(45476, 3)

Primero eliminamos los valores duplicados, ya que nulos no existen


In [16]:
df_credits = df_credits.drop_duplicates()
df_credits.shape

(45439, 3)

Obtenemos `character`, `actors`, `director` de las columnas `cast` y `crew` en dict para poder revisar la data


In [17]:
df_credits["characters"] = df_credits["cast"].apply(
    extract_info, obj="character"
)  # personajes
df_credits["actors"] = df_credits["cast"].apply(extract_info)  # actores

# obtenemos el director
df_credits["director"] = df_credits["crew"].apply(
    extract_info, args=("name", lambda x: x["job"] == "Director")
)

# obtenemos los guionistas
df_credits["crew"] = df_credits["crew"].apply(
    extract_info, args=("name", lambda x: x["job"] != "Director")
)

Reorganizamos las columnas


In [18]:
# Removemos las columnas innecesarias
df_credits = df_credits.drop(columns=["cast"])

# Cambiamos el orden de las columnas
df_credits = df_credits[["id", "characters", "actors", "director", "crew"]]

In [19]:
# Guardamos el dataset limpio
df_credits.to_csv("./data/credits.csv", index=False)

#### `keywords.csv`


In [27]:
# Revisamos la cantidad de columnas y filas
df_keywords.shape

(46419, 2)

Ahora eliminamos los valores duplicados, ya que nulos no existen


In [29]:
df_keywords = df_keywords.drop_duplicates()
df_keywords.shape

(45432, 2)

Obtenemos las `keywords` en dict para poder revisar la data


In [30]:
df_keywords["keywords"] = df_keywords["keywords"].apply(extract_info)

Reorganizamos las columnas


In [31]:
df_keywords.head(2)

,id,keywords
0,862,"jealousy, toy, boy, friendship, friends, rivalry, boy next door, new toy, to..."
1,8844,"board game, disappearance, based on children's book, new home, recluse, gian..."


In [32]:
# Guardamos el dataset limpio
df_keywords.to_csv("./data/keywords.csv", index=False)

#### `movies_metadata.csv`


In [41]:
df_movies_metada.shape

(45466, 24)

Antes de nada eliminamos unos datos raro en el datset


In [46]:
index_corrupted = df_movies_metada[
    df_movies_metada["belongs_to_collection"] == "0.065736"
].index
df_movies_metada = df_movies_metada.drop(index_corrupted)

index_corrupted = df_movies_metada[
    df_movies_metada["belongs_to_collection"] == "2.185485"
].index
df_movies_metada = df_movies_metada.drop(index_corrupted)

index_corrupted = df_movies_metada[
    df_movies_metada["belongs_to_collection"] == "1.931659"
].index
df_movies_metada = df_movies_metada.drop(index_corrupted)

Ahora procesaremos los datos nulos de cada columna (Categorica)


In [ ]:
# belongs_to_collection
df_movies_metada["belongs_to_collection"].fillna(
    "{'name': 'No'}", inplace=True)
df_movies_metada["belongs_to_collection"] = df_movies_metada[
    "belongs_to_collection"
].apply(extract_info)

In [ ]:
# genres
df_movies_metada["genres"].fillna("No", inplace=True)
df_movies_metada["genres"] = df_movies_metada["genres"].apply(extract_info)

In [52]:
# production_companies
df_movies_metada["production_companies"].fillna("No", inplace=True)
df_movies_metada["production_companies"] = df_movies_metada[
    "production_companies"
].apply(extract_info)

In [ ]:
# spoken_languages
df_movies_metada["spoken_languages"].fillna("No", inplace=True)
df_movies_metada["spoken_languages"] = df_movies_metada["spoken_languages"].apply(extract_info)

In [ ]:
# tag_line
df_movies_metada["tagline"].fillna("No", inplace=True)

In [ ]:
# status
df_movies_metada["status"].fillna("No", inplace=True)

In [ ]:
# original_language
df_movies_metada["original_language"].fillna("No", inplace=True)

In [ ]:
# overview
df_movies_metada["overview"].fillna("No", inplace=True)

Ahora procesaremos los datos nulos de cada columna (Numericas)


In [ ]:
# popularity
df_movies_metada["popularity"] = df_movies_metada["popularity"].astype(
    "float"
)  # convert to float

df_movies_metada["popularity"].fillna(
    df_movies_metada["popularity"].median(), inplace=True
)  # clean the nulls

In [ ]:
# release_date
df_movies_metada["release_date"] = pd.to_datetime(
    df_movies_metada["release_date"], errors="coerce"
)  # convert to datetime


df_movies_metada["release_date"].fillna(
    df_movies_metada["release_date"].mode()[0], inplace=True
)  # clean nulls

In [ ]:
# revenue
df_movies_metada["revenue"] = df_movies_metada["revenue"].astype(
    "float"
)  # convert to float

df_movies_metada["revenue"].fillna(
    df_movies_metada["revenue"].median(), inplace=True
)  # clean nulls

In [ ]:
# runtime
df_movies_metada["runtime"] = df_movies_metada["runtime"].astype(
    "float"
)  # convert to float

df_movies_metada["runtime"].fillna(
    df_movies_metada["runtime"].mean(), inplace=True
)  # clean nulls

In [ ]:
# vote_average
df_movies_metada["vote_average"].fillna(
    df_movies_metada["vote_average"].mean(), inplace=True
)  # clean nulls

df_movies_metada["vote_average"] = df_movies_metada["vote_average"].astype(
    "float"
)  # convert to float

In [ ]:
# vote_count
df_movies_metada["vote_count"].fillna(
    df_movies_metada["vote_count"].median(), inplace=True
)  # clean nulls
df_movies_metada["vote_count"] = df_movies_metada["vote_count"].astype(
    "int64"
)  # convert to int

In [70]:
df_movies_metada["id"] = df_movies_metada["id"].astype("int64")

In [71]:
# guardar el dataset limpio
df_movies_metada.to_csv("./data/movies_metadata.csv", index=False)

#### `data_clean.csv`

In [ ]:
# Creamos un dataset de keyword con credits
df_key_cred = pd.merge(df_keywords, df_credits, how="inner", on="id")

In [ ]:
# Ahora creamos un dataset de movie_metada con df_key_cred
df_key_cred_meta = pd.merge(df_key_cred, df_movies_metada, how="inner", on="id")
df_key_cred_meta.info()

In [ ]:
df_key_cred_meta.to_json("./data/data_clean.json", orient="records")

`ratings.csv`

In [12]:
# drop column timestamp
df_ratings.drop(columns=["timestamp"], inplace=True)

In [14]:
set_users = set(df_ratings["userId"])
# de
df_ratings

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


In [19]:
#find user 6834
df_ratings[df_ratings["userId"] == 6834]

,userId,movieId,rating
670223,6834,74,4.0
670224,6834,902,4.0
670225,6834,912,5.0
670226,6834,1197,5.0
670227,6834,2273,5.0
670228,6834,2687,5.0
670229,6834,2706,4.0
670230,6834,3186,5.0
670231,6834,3617,3.0
670232,6834,3987,2.0


In [ ]:
# save the clean dataset
df_ratings.to_csv("./data/ratings.csv", index=False)